## 会员价值评价RFM

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('PYTHON-RFM实战数据.xlsx')
df.head()

,品牌名称,买家昵称,付款日期,订单状态,实付金额,邮费,省份,城市,购买数量
0,CDA数据分析,叫我李2,2019-01-01 00:17:59,交易成功,186,6,上海,上海市,1
1,CDA数据分析,0cyb1992,2019-01-01 00:59:54,交易成功,145,0,广东省,广州市,1
2,CDA数据分析,萝污萌莉,2019-01-01 07:48:48,交易成功,194,8,山东省,东营市,1
3,CDA数据分析,atblovemyy,2019-01-01 09:15:49,付款以后用户退款成功，交易自动关闭,84,0,江苏省,镇江市,1
4,CDA数据分析,小星期鱼,2019-01-01 09:59:33,付款以后用户退款成功，交易自动关闭,74,0,上海,上海市,1


In [3]:
df.订单状态.unique()

array(['交易成功', '付款以后用户退款成功，交易自动关闭'], dtype=object)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28833 entries, 0 to 28832
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   品牌名称    28833 non-null  object        
 1   买家昵称    28833 non-null  object        
 2   付款日期    28833 non-null  datetime64[ns]
 3   订单状态    28833 non-null  object        
 4   实付金额    28833 non-null  int64         
 5   邮费      28833 non-null  int64         
 6   省份      28833 non-null  object        
 7   城市      28832 non-null  object        
 8   购买数量    28833 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 2.0+ MB


### 数据筛选

In [8]:
# 交易成功的数据：布尔掩码索引
df = df.loc[df.订单状态 == '交易成功',:]
df.shape

(27793, 9)

In [11]:
# 关键变量的提取
df = df.loc[:,['买家昵称','付款日期','实付金额']]
df

,买家昵称,付款日期,实付金额
0,叫我李2,2019-01-01 00:17:59.000,186
1,0cyb1992,2019-01-01 00:59:54.000,145
2,萝污萌莉,2019-01-01 07:48:48.000,194
5,重碎叠,2019-01-01 10:00:07.000,197
6,iho_jann,2019-01-01 10:00:08.000,168
...,...,...,...
28828,nhuliy5,2019-06-30 22:22:09.000,144
28829,2ganlin7,2019-06-30 22:33:42.773,216
28830,点大a,2019-06-30 22:36:52.890,231
28831,8陈爷558989,2019-06-30 22:44:38.392,99


### R = 最近一次消费时间-当前提取数据的时间

In [14]:
df.付款日期.describe()

count                          27793
unique                         27283
top              2019-02-17 00:02:42
freq                               5
first            2019-01-01 00:17:59
last      2019-06-30 22:46:22.511000
Name: 付款日期, dtype: object

In [15]:
df.付款日期.max()

Timestamp('2019-06-30 22:46:22.511000')

In [16]:
df.付款日期.min()

Timestamp('2019-01-01 00:17:59')

In [21]:
# 将用户的一天中的消费记录看成是一笔
r = df.groupby('买家昵称')['付款日期'].max().reset_index()   # 最大值指的是最近的一次消费
r.head()

,买家昵称,付款日期
0,.blue_ram,2019-02-04 17:49:34.000
1,.christiny,2019-01-29 14:17:15.000
2,.willn1,2019-01-11 03:46:18.000
3,.托托m,2019-01-11 02:26:33.000
4,0000妮,2019-06-28 16:53:26.458


In [33]:
# 计算R，将2019-7-1看成是观测时间点
r['R'] = (pd.to_datetime('2019-7-1')-r['付款日期']).dt.days
r = r[['买家昵称','R']]
r.head()

,买家昵称,R
0,.blue_ram,146
1,.christiny,152
2,.willn1,170
3,.托托m,170
4,0000妮,2


### F = 订单频次、购买频次

In [38]:
# 提取出用户购买的日期
df['日期标签'] = df['付款日期'].astype(str).str[:10]
df

,买家昵称,付款日期,实付金额,日期标签
0,叫我李2,2019-01-01 00:17:59.000,186,2019-01-01
1,0cyb1992,2019-01-01 00:59:54.000,145,2019-01-01
2,萝污萌莉,2019-01-01 07:48:48.000,194,2019-01-01
5,重碎叠,2019-01-01 10:00:07.000,197,2019-01-01
6,iho_jann,2019-01-01 10:00:08.000,168,2019-01-01
...,...,...,...,...
28828,nhuliy5,2019-06-30 22:22:09.000,144,2019-06-30
28829,2ganlin7,2019-06-30 22:33:42.773,216,2019-06-30
28830,点大a,2019-06-30 22:36:52.890,231,2019-06-30
28831,8陈爷558989,2019-06-30 22:44:38.392,99,2019-06-30


In [39]:
df_1 = df.groupby(['买家昵称','日期标签'])['付款日期'].count().reset_index()
df_1

,买家昵称,日期标签,付款日期
0,.blue_ram,2019-02-04,1
1,.christiny,2019-01-29,1
2,.willn1,2019-01-11,1
3,.托托m,2019-01-11,1
4,0000妮,2019-06-28,1
...,...,...,...
27387,龙魔鬼女,2019-04-19,1
27388,龟mil宝,2019-02-17,1
27389,龟mil宝,2019-06-19,1
27390,！谢鹏逗逼？,2019-06-06,1


In [41]:
df_1.付款日期.value_counts()

1    27014
2      363
3       12
8        1
5        1
4        1
Name: 付款日期, dtype: int64

In [44]:
f = df_1.groupby('买家昵称')['付款日期'].sum().reset_index()
f.columns=['买家昵称','F']
f.head()

,买家昵称,F
0,.blue_ram,1
1,.christiny,1
2,.willn1,1
3,.托托m,1
4,0000妮,1


### M = 总金额

In [46]:
df_2 = df.groupby('买家昵称')['实付金额'].sum().reset_index()
df_2.columns = ['买家昵称','M']
df_2.head()

,买家昵称,M
0,.blue_ram,49
1,.christiny,183
2,.willn1,34
3,.托托m,37
4,0000妮,164


### 合并

In [47]:
rfm = pd.merge(r,f,on='买家昵称')
rfm = pd.merge(rfm,df_2,on='买家昵称')
rfm.head()

,买家昵称,R,F,M
0,.blue_ram,146,1,49
1,.christiny,152,1,183
2,.willn1,170,1,34
3,.托托m,170,1,37
4,0000妮,2,1,164


### 维度打分

In [54]:
rfm['R-score'] = pd.cut(rfm['R'], bins=[0,30,60,90,120,1000000], labels=[5,4,3,2,1], right=False).astype(float)
rfm.head()

,买家昵称,R,F,M,R-score
0,.blue_ram,146,1,49,1.0
1,.christiny,152,1,183,1.0
2,.willn1,170,1,34,1.0
3,.托托m,170,1,37,1.0
4,0000妮,2,1,164,5.0


In [59]:
rfm['F-score']=pd.cut(rfm['F'], bins = [1,2,3,4,5,1000000], labels = [1,2,3,4,5], right=False).astype(float)
rfm['M-score'] = pd.cut(rfm['M'], bins=[0,50,100,150,200,1000000],labels=[1,2,3,4,5],right=False).astype(float)
rfm.head()

,买家昵称,R,F,M,R-score,F-score,M-score
0,.blue_ram,146,1,49,1.0,1.0,1.0
1,.christiny,152,1,183,1.0,1.0,4.0
2,.willn1,170,1,34,1.0,1.0,1.0
3,.托托m,170,1,37,1.0,1.0,1.0
4,0000妮,2,1,164,5.0,1.0,4.0


In [61]:
rfm.F.value_counts()

1     23367
2      1795
3       217
4        33
5         5
15        1
7         1
6         1
Name: F, dtype: int64

In [62]:
rfm['R是否大于均值'] = (rfm['R-score']>rfm['R-score'].mean())*1
rfm['F是否大于均值'] = (rfm['F-score']>rfm['F-score'].mean())*1
rfm['M是否大于均值'] = (rfm['M-score']>rfm['M-score'].mean())*1
rfm.head()

,买家昵称,R,F,M,R-score,F-score,M-score,R是否大于均值,F是否大于均值,M是否大于均值
0,.blue_ram,146,1,49,1.0,1.0,1.0,0,0,0
1,.christiny,152,1,183,1.0,1.0,4.0,0,0,1
2,.willn1,170,1,34,1.0,1.0,1.0,0,0,0
3,.托托m,170,1,37,1.0,1.0,1.0,0,0,0
4,0000妮,2,1,164,5.0,1.0,4.0,1,0,1


In [63]:
rfm['人群数值'] = (rfm['R是否大于均值']*100)+rfm['F是否大于均值']*10+rfm['M是否大于均值']
rfm.head()

,买家昵称,R,F,M,R-score,F-score,M-score,R是否大于均值,F是否大于均值,M是否大于均值,人群数值
0,.blue_ram,146,1,49,1.0,1.0,1.0,0,0,0,0
1,.christiny,152,1,183,1.0,1.0,4.0,0,0,1,1
2,.willn1,170,1,34,1.0,1.0,1.0,0,0,0,0
3,.托托m,170,1,37,1.0,1.0,1.0,0,0,0,0
4,0000妮,2,1,164,5.0,1.0,4.0,1,0,1,101


In [64]:
# 判读RFM
def transform_label(x):
    if x==111:
        label='重要价值客户'
    elif x==11:
        label='重要召回客户'
    elif x==101:
        label='重要发展客户'
    elif x==1:
        label='重要挽留客户'
    elif x==110:
        label='潜力客户'
    elif x==100:
        label='新客户'
    elif x==10:
        label='一般维持客户'
    elif x==0:
        label='流失客户'
    return label

In [65]:
rfm['人群类型'] = rfm['人群数值'].apply(transform_label)
rfm.head()

,买家昵称,R,F,M,R-score,F-score,M-score,R是否大于均值,F是否大于均值,M是否大于均值,人群数值,人群类型
0,.blue_ram,146,1,49,1.0,1.0,1.0,0,0,0,0,流失客户
1,.christiny,152,1,183,1.0,1.0,4.0,0,0,1,1,重要挽留客户
2,.willn1,170,1,34,1.0,1.0,1.0,0,0,0,0,流失客户
3,.托托m,170,1,37,1.0,1.0,1.0,0,0,0,0,流失客户
4,0000妮,2,1,164,5.0,1.0,4.0,1,0,1,101,重要发展客户
